# Long-term

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
#plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
import seaborn as sns
import math
import gc
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
import random

%matplotlib inline

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))


### Model inputs

In the model health states are declared as:
- a0: *susceptible to both breast - and ovarian cancer*
- a1: *susceptible to ovarian cancer only (assumed not susceptible to a second (relative) or third (index) round of bc*
- a2: *susceptible to breast cancer only (survivor of ovarian cancer)*
- b0: *breast cancer diagnosis*
- b1: *breast cancer stage 1*
- b2: *breast cancer stage 2*
- b3: *breast cancer stage 3*
- b4: *breast cancer stage 4*
- c0: *ovarian cancer diagnosis*
- c1: *ovarian cancer local*
- c2: *ovarian cancer regional*
- c3: *ovarian cancer distant*
- d0: *prophylactic masectomy and oophorectomy*
- d1: *prophylactic masectomy only*
- d2: *prophylactic oophorectomy only*
- e0: *no increased risk for cancer in respective organs (following successful d0)*
- f0: *dead from any cause ('normal' mortality + exces mortality due to organ specific cancer)*
- f1: *dead from breast cancer*
- f2: *dead from ovarian cancer*

#### Transition probability sets

In [3]:
from Transitions import *

C:\Users\larsasp\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\common.py:1920: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype)


## Model specification

#### Set starting age

In [4]:
from Controls import initial_age
initial_age

In [5]:
def append_time(time):
    t = time.append(1)
    return t

In [6]:
def time_in_state(time):
    t = len(time)
    return t

In [7]:
from mesa import Agent, Model

class Person(Agent):
    """An agent"""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = initial_age.value
        self.state_a0 = 1
        self.state_a1 = 0
        self.state_a2 = 0
#        self.state_a2r = 0
        self.state_b0 = 0
        self.state_b1 = 0
        self.state_b2 = 0
        self.state_b3 = 0
        self.state_b4 = 0
        self.state_c0 = 0 
        self.state_c1 = 0
        self.state_c2 = 0
        self.state_c3 = 0
        self.state_d0 = 0
        self.state_d1 = 0
        self.state_d2 = 0
        self.state_e0 = 0
        self.state_f0 = 0
        self.state_f1 = 0
        self.state_f2 = 0
        self.t_b1 = []
        self.t_b2 = []
        self.t_b3 = []
        self.t_b4 = []
        self.t_c1 = []
        self.t_c2 = []
        self.t_c3 = []
        self.cycle = 0
        
    def step(self):
        """
        state a0 is the starting state for all agents in self.cycle = 0
        """
        if self.cycle > 0:
            if self.state_a0 == 1:
                if from_a0(self.age)[1] == 1:
                    self.state_b0 += 1
                    self.state_a0 -= 1
                    self.age += 1
                elif from_a0(self.age)[2] == 1:
                    self.state_c0 += 1
                    self.state_a0 -= 1
                    self.age += 1
                elif from_a0(self.age)[3] == 1:
                    self.state_d0 += 1
                    self.state_a0 -= 1
                    self.age += 1
                elif from_a0(self.age)[4] == 1:
                    self.state_d1 += 1
                    self.state_a0 -= 1
                    self.age += 1
                elif from_a0(self.age)[5] == 1:
                    self.state_d2 += 1
                    self.state_a0 -= 1
                    self.age += 1
                elif from_a0(self.age)[6] == 1:
                    self.state_f0 += 1
                    self.state_a0 -= 1
                else:
                    self.state_a0 += 0
                    self.age += 1
                
                
            elif self.state_a1 == 1:
                if from_a1(age = self.age)[0] == 1:
                    self.state_a1 += 0
                    self.age += 1
                elif from_a1(age = self.age)[1] == 1:
                    self.state_c0 += 1
                    self.state_a1 -= 1
                    self.age += 1
                elif from_a1(age = self.age)[2] == 2:
                    self.state_d2 += 1
                    self.state_a1 -= 1
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_a1 -= 1
                        
            elif self.state_a2 == 1:
                if from_a2(age = self.age)[0] == 1:
                    self.state_a2 += 0
                    self.age += 1
                elif from_a2(age = self.age)[1] == 1:
                    self.state_b0 += 1
                    self.state_a2 -= 1
                    self.age += 1
                elif from_a2(age = self.age)[3] == 1:
                    self.state_d1 += 1
                    self.state_a2 -= 1
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_a2 -= 1
                        
#                if self.state_a2r == 1:
#                    if from_a2r(age = self.age)[0] == 1:
#                        self.state_a2 += 0
#                        self.age += 1
#                    if from_a2r(age = self.age)[1] == 1:
#                        self.state_b0 += 1
#                        self.state_a2 -= 1
#                        self.age += 1
#                    if from_a2r(age = self.age)[3] == 1:
#                        self.state_d1 += 1
#                        seøf.state_a2 -= 1
#                        self.age += 1
#                    else:
#                        self.state_f0 += 1
#                        self.state_a2 -= 1

            elif self.state_b0 == 1:
                if from_b0(age = self.age)[0] == 1:
                    self.state_b1 += 1
                    self.state_b0 -= 1
                elif from_b0(age = self.age)[1] == 1:
                    self.state_b2 += 1
                    self.state_b0 -= 1
                elif from_b0(age = self.age)[2] == 1:
                    self.state_b3 += 1
                    self.state_b0 -= 1
                else:
                    self.state_b4 += 1
                    self.state_b0 -= 1
                        
            elif self.state_b1 == 1:
                if from_b1(self.age, time_in_state(self.t_b1))[1] == 1:
                    self.state_f1 += 1
                    self.state_b1 -= 1
                elif from_b1(self.age, time_in_state(self.t_b1))[2] == 1:
                    self.state_a1 += 1
                    self.state_b1 -= 1
                    self.age += 1
                else:
                    append_time(self.t_b1)
                    self.state_b1 += 0                    
                    self.age += 1
                        

            elif self.state_b2 == 1:
                if from_b2(self.age, time_in_state(self.t_b2))[1] == 1:
                    self.state_f1 += 1
                    self.state_b2 -= 1          
                elif from_b2(self.age, time_in_state(self.t_b2))[2] == 1:
                    self.state_a1 +=1
                    self.state_b2 -=1
                    self.age += 1
                else:
                    append_time(self.t_b2)
                    self.state_b2 += 0
                    self.age += 1


            elif self.state_b3 == 1:
                if from_b3(self.age, time_in_state(self.t_b3))[1] == 1:
                    self.state_f1 += 1
                    self.state_b3 -= 1          
                elif from_b3(self.age, time_in_state(self.t_b3))[2] == 1:
                    self.state_a1 +=1
                    self.state_b3-=1
                    self.age += 1
                else:
                    append_time(self.t_b3)
                    self.state_b3 += 0
                    self.age += 1
                        
                        
            elif self.state_b4 == 1:
                if from_b4(self.age, time_in_state(self.t_b4))[1] == 1:
                    self.state_f1 += 1
                    self.state_b4 -= 1          
                elif from_b4(self.age, time_in_state(self.t_b4))[2] == 1:
                    self.state_a1 +=1
                    self.state_b4 -=1
                    self.age += 1
                else:
                    append_time(self.t_b4)
                    self.state_b4 += 0
                    self.age += 1
                     
      
            elif self.state_c0 == 1:
                if from_c0()[0] == 1:
                    self.state_c1 += 1
                    self.state_c0 -= 1
                elif from_c0()[1] == 1:
                    self.state_c2 += 1
                    self.state_c0 -= 1
                else:
                    self.state_c3 += 1
                    self.state_c0 -= 1
                        
                
            elif self.state_c1 == 1:
                if from_c1(self.age, time_in_state(self.t_c1))[1] == 1:
                    self.state_f2 += 1
                    self.state_c1 -= 1
                elif from_c1(self.age, time_in_state(self.t_c1))[2] == 1:
                    self.state_a2 += 1
                    self.state_c1 -=1
                    self.age +=1
                else:
                    append_time(self.t_c1)
                    self.state_c1 += 0
                    self.age +=1
                    
                        
            elif self.state_c2 == 1:
                if from_c2(self.age, time_in_state(self.t_c2))[1] == 1:
                    self.state_f2 += 1
                    self.state_c2 -= 1
                elif from_c2(self.age, time_in_state(self.t_c2))[2] == 1:
                    self.state_a2 += 1
                    self.state_c2 -=1
                    self.age +=1
                else:
                    append_time(self.t_c2)
                    self.state_c2 += 0
                    self.age +=1
                
            elif self.state_c3 == 1:
                if from_c3(self.age, time_in_state(self.t_c3))[1] == 1:
                    self.state_f2 += 1
                    self.state_c3 -= 1
                elif from_c3(self.age, time_in_state(self.t_c3))[2] == 1:
                    self.state_a2 += 1
                    self.state_c3 -=1
                    self.age +=1
                else:
                    append_time(self.t_c3)
                    self.state_c3 += 0
                    self.age +=1
                        
            elif self.state_d0 == 1:
                if from_d0(age = self.age) == 1:
                    self.state_d0 += 0
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_d0 -= 1
                        
            elif self.state_d1 == 1:
                if from_d1(age = self.age) == 1:
                    self.state_d1 += 0
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_d1 -= 1        
                        
            elif self.state_d2 == 1:
                if from_d2(age = self.age) == 1:
                    self.state_d2 += 0
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_d2 -= 1
                        
            elif self.state_e0 == 1:
                if from_e0(age = self.age) == 1:
                    self.state_e0 += 0
                    self.age += 1
                else:
                    self.state_f0 += 1
                    self.state_e0 -= 1
                        
        self.cycle += 1

In [8]:
def trace_a0(model):
    agent_a0 = [agent.state_a0 for agent in model.schedule.agents]
    x = sum(agent_a0)
    N = model.num_agents
    B = x / N
    return x

def trace_a1(model):
    agent_a1 = [agent.state_a1 for agent in model.schedule.agents]
    x = sum(agent_a1)
    N = model.num_agents
    B = x / N
    return x

def trace_a2(model):
    agent_a2 = [agent.state_a2 for agent in model.schedule.agents]
    x = sum(agent_a2)
    N = model.num_agents
    B = x / N
    return x

#def trace_A2r(model):
#    agent_a2r = [agent.state_A2r for agent in model.schedule.agents]
#    x = sum(agent_a2r)
#    N = model.num_agents
#    B = x / N
#    return B

def trace_b1(model):
    agent_b1 = [agent.state_b1 for agent in model.schedule.agents]
    x = sum(agent_b1)
    N = model.num_agents
    B = x / N
    return x

def trace_b2(model):
    agent_b2 = [agent.state_b2 for agent in model.schedule.agents]
    x = sum(agent_b2)
    N = model.num_agents
    B = x / N
    return x

def trace_b3(model):
    agent_b3 = [agent.state_b3 for agent in model.schedule.agents]
    x = sum(agent_b3)
    N = model.num_agents
    B = x / N
    return x

def trace_b4(model):
    agent_b4 = [agent.state_b4 for agent in model.schedule.agents]
    x = sum(agent_b4)
    N = model.num_agents
    B = x / N
    return x

def trace_c1(model):
    agent_c1 = [agent.state_c1 for agent in model.schedule.agents]
    x = sum(agent_c1)
    N = model.num_agents
    B = x / N
    return x

def trace_c2(model):
    agent_c2 = [agent.state_c2 for agent in model.schedule.agents]
    x = sum(agent_c2)
    N = model.num_agents
    B = x / N
    return x

def trace_c3(model):
    agent_c3 = [agent.state_c3 for agent in model.schedule.agents]
    x = sum(agent_c3)
    N = model.num_agents
    B = x / N
    return x

def trace_e0(model):
    agent_e0 = [agent.state_e0 for agent in model.schedule.agents]
    x = sum(agent_e0)
    N = model.num_agents
    B = x / N
    return x

def trace_f0(model):
    agent_f0 = [agent.state_f0 for agent in model.schedule.agents]
    x = sum(agent_f0)
    N = model.num_agents
    B = x / N
    return x

def trace_f1(model):
    agent_f1 = [agent.state_f1 for agent in model.schedule.agents]
    x = sum(agent_f1)
    N = model.num_agents
    B = x / N
    return x

def trace_f2(model):
    agent_f2 = [agent.state_f2 for agent in model.schedule.agents]
    x = sum(agent_f2)
    N = model.num_agents
    B = x / N
    return x


In [9]:
class hboc(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = N
        self.cycle  = 0
        self.schedule = RandomActivation(self)
        
        # Create agents
        for i in range(self.num_agents):
            a = Person(i, self)
            self.schedule.add(a)    
        # Store data after each step
        self.datacollector = DataCollector(model_reporters=
                                           {"a0": trace_a0,
                                            "a1": trace_a1,
                                            "a2": trace_a2,
                                            "b1": trace_b1,
                                            "b2": trace_b2,
                                            "b3": trace_b3,
                                            "b4": trace_b4,
                                            "c1": trace_c1,
                                            "c2": trace_c2,
                                            "c3": trace_c3,
                                            "f1": trace_f0,
                                            "f2": trace_f1,
                                            "f3": trace_f2},
                                          agent_reporters={
                                             "a0": lambda a: a.state_a0,
                                             "a1": lambda a: a.state_a1,
                                             "a2": lambda a: a.state_a2,
                                             "b1": lambda a: a.state_b1,
                                             "b2": lambda a: a.state_b2,
                                             "b3": lambda a: a.state_b3,
                                             "b4": lambda a: a.state_b4,
                                             "c1": lambda a: a.state_c1,
                                             "c2": lambda a: a.state_c2,
                                             "c3": lambda a: a.state_c3,
                                             "f0": lambda a: a.state_f0,
                                             "f1": lambda a: a.state_f1,
                                             "f2": lambda a: a.state_f2}
                                          )
        
    def step(self):
        # store data  
        self.datacollector.collect(self)
                
        #advnace a step
        self.schedule.step()

## Run

In [10]:
%%time
model = hboc(1000)
for i in range(51):
    model.step()


Wall time: 39.6 s


## Agent-level output

In [11]:
all_agent_movement = model.datacollector.get_agent_vars_dataframe()
all_agent_movement.to_csv('Agents.csv')
df = pd.read_csv('Agents.csv')
df

,Step,AgentID,b4,a0,f2,c2,a1,b3,c1,f1,f0,a2,b2,c3,b1
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,4,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,5,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,6,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,7,0,1,0,0,0,0,0,0,0,0,0,0,0
8,0,8,0,1,0,0,0,0,0,0,0,0,0,0,0
9,0,9,0,1,0,0,0,0,0,0,0,0,0,0,0


In [12]:
df2 = df.groupby('AgentID')['a0','a1', 'a2', 'b1', 'b2', 'b3', 'b4', 'c1', 'c2', 'c3', 'f0', 'f1', 'f2'].sum()
df2

,a0,a1,a2,b1,b2,b3,b4,c1,c2,c3,f0,f1,f2
AgentID,,,,,,,,,,,,,
0,29,0,0,0,0,0,0,0,0,0,22,0,0
1,34,0,0,0,0,0,0,0,0,0,17,0,0
2,19,0,0,0,0,0,0,0,0,0,32,0,0
3,18,0,0,0,0,0,0,0,0,0,33,0,0
4,2,0,0,0,0,0,0,0,0,0,49,0,0
5,41,0,0,0,0,0,0,0,0,0,10,0,0
6,7,0,0,0,0,0,0,0,0,0,44,0,0
7,16,0,0,0,0,0,0,0,0,0,35,0,0
8,9,0,0,0,0,0,0,0,0,0,42,0,0


### Check: how long in breast cancer state if impacted? (Should not exceed 5)

In [13]:
b = df2[((df2.b1 > 0) & (df2.a1 > 0)) | (((df2.b2 > 0) & (df2.a1 > 0))) | (((df2.b3 > 0) & (df2.a1 > 0))) | ((df2.b4 > 0) & (df2.a1 > 0))]

In [14]:
b

,a0,a1,a2,b1,b2,b3,b4,c1,c2,c3,f0,f1,f2
AgentID,,,,,,,,,,,,,
11,10,24,0,5,0,0,0,0,0,0,11,0,0
30,9,13,0,5,0,0,0,0,0,0,23,0,0
32,17,8,0,0,5,0,0,0,0,0,20,0,0
57,19,12,0,5,0,0,0,0,0,0,14,0,0
65,11,1,0,0,5,0,0,0,0,0,33,0,0
67,2,3,0,5,0,0,0,0,0,0,40,0,0
70,4,31,0,5,0,0,0,0,0,0,10,0,0
78,5,19,0,5,0,0,0,0,0,0,21,0,0
97,15,4,0,5,0,0,0,0,0,0,26,0,0


In [15]:
b.b1.value_counts()

5    95
0    47
Name: b1, dtype: int64

In [16]:
b.b2.value_counts()

0    108
5     34
Name: b2, dtype: int64

In [17]:
b.b3.value_counts()

0    142
Name: b3, dtype: int64

In [18]:
b.b4.value_counts()

0    129
5     13
Name: b4, dtype: int64

### Check: how long in ovarian cancer state if impacted? (Should not exceed 10)

In [24]:
c = df2[(df2.c1 > 0) | (df2.c2 > 0) | (df2.c3 > 0)]
c

,a0,a1,a2,b1,b2,b3,b4,c1,c2,c3,f0,f1,f2
AgentID,,,,,,,,,,,,,
187,5,0,0,0,0,0,0,0,0,1,0,0,44
341,22,0,9,0,0,0,0,0,9,0,10,0,0
399,33,0,0,0,0,0,0,0,0,5,0,0,12
471,15,0,0,0,0,0,0,0,0,1,0,0,34
551,20,0,0,0,0,0,0,0,0,3,0,0,27
725,24,0,1,0,0,0,0,9,0,0,16,0,0
738,15,0,0,0,0,0,0,0,0,3,0,0,32
744,14,0,8,0,0,0,0,0,9,0,19,0,0
748,12,0,0,0,0,0,0,0,0,2,0,0,36


In [25]:
c.c1.value_counts()

0    9
9    1
Name: c1, dtype: int64

In [26]:
c.c2.value_counts()

0    8
9    2
Name: c2, dtype: int64

In [27]:
c.c3.value_counts()

1    3
0    3
3    2
5    1
2    1
Name: c3, dtype: int64

## Model-level output

In [28]:
output = model.datacollector.get_model_vars_dataframe()
output

,a0,a1,a2,b1,b2,b3,b4,c1,c2,c3,f1,f2,f3
0,1000,0,0,0,0,0,0,0,0,0,0,0,0
1,1000,0,0,0,0,0,0,0,0,0,0,0,0
2,948,0,0,0,0,0,0,0,0,0,36,0,0
3,912,0,0,10,2,0,4,0,0,0,59,0,0
4,872,0,0,18,4,0,5,0,0,0,90,2,0
5,831,0,0,21,8,0,6,0,0,0,120,3,0
6,788,0,0,27,6,0,8,0,0,1,152,7,0
7,756,0,0,30,9,0,11,0,0,0,173,9,1
8,713,11,0,25,11,0,13,0,0,0,207,10,1
9,666,24,0,23,9,0,10,0,0,0,239,13,1


In [29]:
# Det ser kanskje ut som om koden har en bug som gjør at funksjonen med "dersom i state_bi" i 10 år -> a1, 
# den fungerer faktisk, men i o m at denne kreften er såpass sjelden på aggregert nivå, og at den har en økt dødelighet
# ser det slik ut. 

In [30]:
ax = output.plot()
ax.set_xlabel('Years')
ax.set_ylabel('% in state')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title('HBOC')
#plt.savefig('M:\pc\Desktop\plot.png', dpi = 400)